# PartII Neural Networks and Deep Learning
## Chapter 9 Up and running with TensorFlow


＜参考＞

- ①最急降下法、一変数の最適化問題
http://shogo82148.github.io/homepage/memo/algorithm/steepest-descent/
- ②最急降下法のイメージと例　https://mathwords.net/saikyukouka
- ③最急降下法の図解
https://github.com/levelfour/machine-learning-2014/wiki/%E7%AC%AC4%E5%9B%9E---%E5%8B%BE%E9%85%8D%E6%B3%95%EF%BC%88%E6%9C%80%E6%80%A5%E9%99%8D%E4%B8%8B%E6%B3%95%EF%BC%89
    

### [9-9] Visualizing the graph and training curves using tensorboard p.254


- ミニバッチ勾配降下を使用して線形回帰モデルを訓練する計算グラフを作成し、定期的にチェックポイントを保存。
- TensorBoardは、統計情報をフィード
- ウェブブラウザに統計情報をインタラクティブに表示可能
- 例えば学習曲線を表示
- グラフのエラーを特定したり、ボトルネックなどを発見するのに便利。
- 最初のステップで、TensorBoardが読み込むログディレクトリ生成。
- グラフ定義といくつかのトレーニングの統計情報、トレーニングエラー（MSE）を書き出す。- プログラムを実行するたびに別のログディレクトリが必要
- 別のログディレクトリを用意しないなら、TensorBoardは実行毎に統計情報をマージしてしまう。
- 最も簡単な解決策は、タイムスタンプをログディレクトリ名にする事。



＜Tensorboard設定①＞
プログラムの最初に、ログディレクトリを指定するコードを入れておく

In [11]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

＜Tensorboard設定②＞
Construction Phaseの最後に、次のコードを入れる。

mse_summary = tf.summary.scalar('MSE', mse) #教科書は古い tf.scalar_summary[deprecated]
summary_writer = tf.train.SummaryWriter(logdir, tf.get_default_graph()) #tf.summary.filewriter

- 1行目は、MSE値を評価し、TensorBoard互換のバイナリログ文字列（要約）に書き込むノードを作成する。
- 2行目はログディレクトリのログファイルに要約を書き込むSummaryWriterを生成。最初のパラメータはログディレクトリパス、2番目のoptionのパラメータは視覚化対象のgraph。
- 新規ディレクトリは生成。
- eventファイル=グラフ定義が書かれたバイナリログファイル

＜Tensorboard設定③＞

実行フェーズでmse_summary.eval（たとえば10個のミニバッチ毎に）。 

[...]
for batch_index in range(n_batches):
X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
if batch_index % 10 == 0:
summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
step = epoch * n_batches + batch_index
summary_writer.add_summary(summary_str, step)
sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
[...]

- logを1学習ステップ毎に取ると計算スピードが遅くなる

＜Tensorboard設定④＞
summary_writer.close()

- プログラムを実行すると、ログディレクトリ作成、グラフ定義とMSE値の両方を含むイベントファイルが書き込まれる
- 作業ディレクトリに移動し、ls -l tf_logs/run* と入力してログディレクトリの内容を表示
- $ cd $ML_PATH # Your ML working directory (eg. $HOME/ml)
  $ ls -l tf_logs/run*
  total 40
  -rw-r--r-- 1 ageron staff 18620 Sep 6 11:10 events.out.tfevents.1472553182.mymac
  Visualizing the

In [ ]:
＜Tensorboardを動かす：①起動＞
tensorboard serverを起動

$ source env/bin/activate　#仮想環境activate
$ tensorboard --logdir tf_logs/ #tensorboard web server起動 
Starting TensorBoard on port 6006
(ブラウザでhttp://0.0.0.0:6006を開く)

＜Tensorboardを動かす：②Event Tab＞
- Event tab :  MSEが右側にグラフ表示
- 学習、実行どちらでも図確認可能、ズーム機能等。

In [ ]:
＜Tensorboardを動かす：③Graphs Tab＞
- 多くのエッジを持つノードは、clutterを減らす為に、右側の補助領域に分割表示される
- メイングラフと補助領域の間でノードを右クリックで移動
- グラフの一部はデフォルトで折りたたまれている。
-- 1)グラデーションノードの⊕アイコンクリックで、サブグラフを展開。
-- 2)サブグラフでmse_gradサブグラフが展開


＜Jupyter内でグラフを直接見る方法＞
①show_graph（）関数使用＝チュートリアルノート（http://goo.gl/EtCWUc）by  A. Mordvintsev
②TensorFlowデバッガツール（https://github.com/ericjang/tdb） by E. Jang

In [16]:
#- 作図用の関数を準備。[MatplotLib plots figures inline/Saving figures]
#- ホーム直下のimagesディレクトリを作成しておく。

# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

#----- Common imports
import numpy as np
import os

#--------- to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

#----- To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

#-------- Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [17]:
import tensorflow as tf
import tensorboard as tb   

#default_graphをckptから読込
reset_graph()

saver = tf.train.import_meta_graph("./tmp/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # not shown in the book

with tf.Session() as sess:
    saver.restore(sess, "./tmp/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from ./tmp/my_model_final.ckpt


In [18]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

Firefoxで動かない。Google chromeで画像が出る。

In [19]:
show_graph(tf.get_default_graph())

### [9-9-3] Name scopes p.257
ニューラルネットワークの様な複雑なモデルを扱う場合、グラフは数千ノードで煩雑になる場合がある。 
これを避けるには、Name Scopeを使用して関連ノードをグループ化する。 

上記のコードを変更して"loss"というNameScope内でエラーとmse操作を定義しましょう。

In [21]:
reset_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "./tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01
n = n_epochs ### modification

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

In [22]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [23]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [26]:
# m, fetch_batchの定義が無い
datasize = 1000
m = datasize
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))


#def fetch_batch(epoch, batch_index, batch_size):
#    rnd.seed(epoch * n_batches + batch_index)
#    indices = rnd.randint(m, size=batch_size)
#    X_batch = scaled_housing_data_plus_bias[indices]
#    y_batch = housing.target.reshape(-1, 1)[indices]
#    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

NameError: name 'rnd' is not defined

スコープ内で定義されている各演算子Opの名前は、先頭に "loss /"が付きます。

In [27]:
print(error.op.name)

loss/sub


In [28]:
print(mse.op.name)

loss/mse


TensorBoardでは、デフォルトで折りたたまれているLoss Name Spaceの中に、
mseノードとエラーノードがある。
(Figure 9-5).デフォルトで折りたたまれている。

In [29]:
# 本に掲載が無いコード
reset_graph()

a1 = tf.Variable(0, name="a")      # name == "a"
a2 = tf.Variable(0, name="a")      # name == "a_1"

with tf.name_scope("param"):       # name == "param"
    a3 = tf.Variable(0, name="a")  # name == "param/a"

with tf.name_scope("param"):       # name == "param_1"
    a4 = tf.Variable(0, name="a")  # name == "param_1/a"

for node in (a1, a2, a3, a4):
    print(node.op.name)

a
a_1
param/a
param_1/a


### [9-9-4] Modularity p.258

2つのRectified Linear Units（ReLU）の出力を加算するグラフを作成したいとする。 ReLUは、入力値の線形関数を計算し、正の場合は入力結果を出力とし、それ以外は0を出力。

Equation 9-1. Rectified Linear Unit (ReLU)
$$
h_{w,b}(X)= max (X·w+ b, 0)
$$

-------------------------------------------------------
参考：活性化関数とは
- ①高卒でもわかる機械学習　http://hokuts.com/2016/05/27/pre-bp/
- ②今日から始めるDeep　Learning
https://developers.eure.jp/tech/start_deep_learning_today/
------------------------------------------------------

参考：活性化関数(z=f(x)）リスト

- ①ステップ関数
- ②sigmoid関数：2値分類出力層
- ③正規化線形関数、ランプ関数ReLU、
- ④恒等関数：回帰問題出力層
- ⑤ソフトマックス関数：分類問題出力層

https://ja.wikipedia.org/wiki/%E6%B4%BB%E6%80%A7%E5%8C%96%E9%96%A2%E6%95%B0
https://qiita.com/namitop/items/d3d5091c7d0ab669195f

次のコードは動くが、繰り返しのみ。


In [30]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

- このような反復コードは維持しにくく、エラーを起こしやすい
（実際に、上記コードにはCut&Pasteのエラーが含まれている）。 

- TensorflowはReLU関数を持つ
- 次コードはReLUsを計算し、合計を出力する。
- add_n（）はテンソルのリストの和

In [31]:
reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [32]:
file_writer = tf.summary.FileWriter("./logs/relu1", tf.get_default_graph())

- ノードを作成すると、TensorFlowはその名前が既に存在するかどうかをチェック
- 名前があればアンダースコアとそれに続くインデックスを付けて名前を作成

（Figure 9-6）
- 最初のReLU="weights"、 "bias"、 "z"、 "relu"
- 2番目ReLU="weights_1"、 "bias_1"などのノード
- 3番目ReLU="weights_2"、 "bias_2"などのノード

Even better using name scopes:
以下は本に含まれないコード


In [33]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    # not shown
        b = tf.Variable(0.0, name="bias")                             # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                      # not shown
        return tf.maximum(z, 0., name="max")                          # not shown

In [34]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("./logs/relu2", tf.get_default_graph())
file_writer.close()

### [9-9-5] Sharing Variables　p.261

- グラフのコンポーネント間で変数を共有したい場合は、最初に変数を作成し、その関数を必要とする関数にパラメータとして渡す。
- たとえば、すべてのReLUの閾値を変数を考える。 
- 一つのthrehold閾値変数で全relu（）関数を制御したい。

- 変数を作ってrelu()に通す

In [35]:
reset_graph()

def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

- このような共有パラメータが多数ある場合は、常にパラメータとして渡す必要があります。 - 多くの人は、モデル内のすべての変数を含むPython辞書を作成し、すべての関数に渡します。 
- 他のモジュールは、各モジュールのクラスを作成します
- さらにもう1つの方法は、最初の呼び出し時に共有変数をrelu（）関数の属性として設定することです：

In [36]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, relu.threshold, name="max")

In [85]:
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

- TensorFlowには別のオプションがある。
- この解決策は最初は理解するのが少し難解ですが、それがTensorFlowで多く使用されている
- アイデアは、共有変数が存在しない場合は、get_variable（）関数を使用して共有変数を作成すること 

- 目的の動作（作成または再利用）は、現在のvariable_scope（）の属性によって制御
- 例えば次コードは "relu / threshold"という名前の変数を作成
-（shape =（）はスカラー、初期値として0.0を使用）


In [86]:
reset_graph()

with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))

- get_variable（）の以前の呼び出しによって変数がすでに作成されている場合、このコードは例外を発生させます。 
- この動作は、誤って変数を再利用することを防ぎます。
- 変数を再利用する場合は、変数スコープのreuse属性をTrueに設定することで、明示する必要があります（その場合は、シェイプまたはイニシャライザを指定する必要はありません）。

In [87]:
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

- このコードは、既存の "relu / threshold"変数をフェッチします
-（存在しない場合、またはget_variable（）を使用して作成されていない場合は例外を発生させます）。 
- 代わりに、スコープのreuse_variables（）メソッドを呼び出すことによって、ブロック内でreuse属性をTrueに設定することもできます。

In [88]:
with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

In [89]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

- このコードでは最初にrelu（）関数を定義し、
- relu / threshold変数（0.0に初期化されるスカラー）を作成し、
- relu（）関数を呼び出して5 ReLUsを作成します。 
- relu（）関数は、relu / threshold変数を再利用し、他のReLUノードを作成します。

In [90]:
file_writer = tf.summary.FileWriter("logs/relu6", tf.get_default_graph())
file_writer.close()

- 閾値変数はrelu（）関数の外側で定義しなければならない。
- これを修正するには、以下のコードは、最初の呼び出し時にrelu（）関数内に
閾値変数を作成し、その後の呼び出しでそれを再利用します。

- 名前スコープや変数共有について：get_variable（）を呼び出すだけで、
閾値変数を作成または再利用します（どちらが該当するかを知る必要はありません）。 
- 残りのコードは、relu（）を5回呼び出します。最初の呼び出しでは
reuse = Falseを設定し、他の呼び出しではreuse = Trueを設定します。

In [91]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu"):
        threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("", default_name="") as scope:
    first_relu = relu(X)     # create the shared variable
    scope.reuse_variables()  # then reuse it
    relus = [first_relu] + [relu(X) for i in range(4)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("/linux/logs/relu8", tf.get_default_graph())
file_writer.close()

In [87]:
reset_graph()

def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

The resulting graph is slightly different than before, since the shared variable lives within the first ReLU (see Figure 9-9).

In [92]:
file_writer = tf.summary.FileWriter("/linux/logs/relu9", tf.get_default_graph())
file_writer.close()

- 次章、特に次の操作について説明

 - 畳み込みニューラルネットワーク、
 - リカレントニューラルネットワーク、
 - マルチスレッド、キュー、複数のGPU、複数のサーバーを使用してTensorFlowでスケールアップする方法